# Analyzing online sports revenue
Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade.

In this notebook, the clothing company is specifically interested in how it can improve revenue. The main units of analysis are pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

The project comes with 4 datasets to analyse:

###  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

### finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

### info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

### reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

The company asked the following questions: 
1. What is the volume of products and average revenue for Adidas and Nike products based on listing price quartiles?
- Label products priced up to quartile one as "Budget", quartile two as "Average", quartile three as "Expensive", and quartile four as "Elite".
- Store as a pandas DataFrame called adidas_vs_nike containing the following columns: "brand", "price_label", "num_products", and "mean_revenue". All numeric values should be rounded to two decimal places.
2. Do any differences exist between the word count of a product's description and its mean rating?
- Split product description length into bins of 100 words and calculate the average rating and number of reviews.
- Store the results as a pandas DataFrame called description_lengths containing the following columns: "description_length", "mean_rating", "num_reviews", again rounding numeric values to two decimal places.
3. How does the volume of products and median revenue vary between clothing and footwear?
- Search "description" for "shoe*", "trainer*", or "foot*" and use the results to calculate the number of footwear products versus clothing products sold by the company and the median revenue for each of the two product types.
- Create a pandas DataFrame called product_types containing the following columns: "num_clothing_products", "median_clothing_revenue", "num_footwear_products", "median_footwear_revenue".

In [2]:
# load data
import pandas as pd

brands = pd.read_csv("data/brands.csv") 
finance = pd.read_csv("data/finance.csv")
info = pd.read_csv("data/info.csv")
reviews = pd.read_csv("data/reviews.csv")

In [9]:
finance_clean = finance.dropna()

# Get the maximum listing price using .loc
max_price = finance.loc[:, "listing_price"].max()

# Use .loc for creating the 'price_label' column
finance_clean.loc[:, 'price_label'] = ["Budget" if x <= (max_price / 4) else "Average" if x <= (max_price / 3) else "Expensive" if x <= (max_price / 2) else "Elite" for x in finance_clean.loc[:, 'listing_price']]

C:\Users\celin\AppData\Local\Temp\ipykernel_14004\2961238897.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finance_clean['price_label'] = finance_clean['listing_price'].apply(categorize_price)


In [10]:
adidas_vs_nike = pd.merge(
    brands,
    finance_clean[["product_id","revenue","price_label"]],
    how="inner",
    on="product_id",
)

In [11]:
adidas_vs_nike["num_products"] = adidas_vs_nike.groupby(["brand","price_label"])["price_label"].transform('count')

In [12]:
adidas_vs_nike["mean_revenue"] = adidas_vs_nike.groupby(["brand","price_label"])["revenue"].transform('mean').round(2)

In [13]:
import nltk
from nltk.tokenize import word_tokenize
import string
import re

nltk.download('wordnet')
nltk.download('punkt')

def preprocess_text(text):
    # Tokenize the text
    word_tokens = word_tokenize(text)

    # Lowercase each word in the text
    word_tokens = [word.lower() for word in word_tokens]

    # Remove punctuation and numbers
    word_tokens = [re.sub('[^a-zA-Z]', '', word) for word in word_tokens]

    return " ".join(word_tokens)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\celin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\celin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
# Apply the preprocessing function to your text data, handling missing values
info['processed'] = info['description'].fillna('').apply(preprocess_text)

In [19]:
description_lengths = info[["product_name", "product_id"]]
description_lengths.loc[:, "description_length"] = [len(x.split()) for x in info.loc[:, 'processed'].tolist()]


C:\Users\celin\AppData\Local\Temp\ipykernel_14004\2006833182.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  description_lengths.loc[:, "description_length"] = [len(x.split()) for x in info.loc[:, 'processed'].tolist()]


In [20]:
description_lengths = description_lengths.merge(reviews[["product_id","reviews"]], how="inner")

In [22]:
# Merge the data
merged_df = info.merge(finance, on="product_id", how="outer")
merged_df = merged_df.merge(reviews, on="product_id", how="outer")
merged_df = merged_df.merge(brands, on="product_id", how="outer")

# Drop null values
merged_df.dropna(inplace=True)

# Add price labels based on listing_price quartiles
merged_df["price_label"] = pd.qcut(merged_df["listing_price"], q=4, labels=["Budget", "Average", "Expensive", "Elite"])

# Group by brand and price_label to get volume and mean revenue
adidas_vs_nike = merged_df.groupby(["brand", "price_label"], as_index=False).agg(
    num_products=("price_label", "count"), 
    mean_revenue=("revenue", "mean")
).round(2).reset_index(drop=True)

# Find the largest description_length
max(merged_df["description"].str.len())

# Store the length of each description
merged_df["description_length"] = merged_df["description"].str.len()

# Upper description length limits
lengthes = [0, 100, 200, 300, 400, 500, 600, 700]

# Description length labels
labels = ["100", "200", "300", "400", "500", "600", "700"]

# Cut into bins
merged_df["description_length"] = pd.cut(merged_df["description_length"], bins=lengthes, labels=labels)

# Group by the bins
description_lengths = merged_df.groupby("description_length", as_index=False).agg(
    mean_rating=("rating", "mean"), 
    num_reviews=("reviews", "count")
).round(2)

# List of footwear keywords
mylist = "shoe*|trainer*|foot*"

# Filter for footwear products
shoes = merged_df[merged_df["description"].str.contains(mylist)]

# Filter for clothing products
clothing = merged_df[~merged_df.isin(shoes["product_id"])]

# Remove null product_id values from clothing DataFrame
clothing.dropna(inplace=True)

# Create product_types DataFrame
product_types = pd.DataFrame({"num_clothing_products": len(clothing), 
                              "median_clothing_revenue": clothing["revenue"].median(), 
                              "num_footwear_products": len(shoes), 
                              "median_footwear_revenue": shoes["revenue"].median()}, 
                              index=[0])

In [23]:
product_types

,num_clothing_products,median_clothing_revenue,num_footwear_products,median_footwear_revenue
0,478,625.07,2639,3073.3
